In [2]:
import numpy as np
import pandas as pd
import math
import scipy
import sys
from tqdm import tqdm
import functools
import networkx as nx
import itertools

import portion as P
import re
import datetime

# interval = P.empty()
# for i, j in [(0, 1), (2, 4), (6, 7), (3, 5)]:
#     interval = interval | P.closed(i, j)
# print(interval)

In [94]:
class Valve():
    def __init__(self, name, rate = -1, neighbors = None) -> None:
        self.name = name
        self.rate = rate
        self.neighbors = neighbors
    
    def __repr__(self) -> str:
        return f'{self.name}[{self.rate}]'
        
    # def __str__(self) -> str:
    #     return self.name

In [95]:
file1 = open('16.txt', 'r')
lines = file1.readlines()

valves = {}

for l in lines:
    l = l.replace('\n','')
    valve_names = re.findall(r'[A-Z]{2}', l)
    rate = int(re.findall(r'\d+', l)[0])

    if valve_names[0] in valves:
        valves[valve_names[0]].rate = rate
    else:
        valves[valve_names[0]] = Valve(valve_names[0], rate)
    for neigh in valve_names[1:]:
        if neigh not in valves:
            valves[neigh] = Valve(neigh)
    valves[valve_names[0]].neighbors = [valves[neigh] for neigh in valve_names[1:]]

G=nx.Graph()
G.add_nodes_from(valves.values())

In [96]:
valves

{'OK': OK[0],
 'RW': RW[14],
 'FX': FX[16],
 'JY': JY[13],
 'TT': TT[0],
 'LF': LF[0],
 'GO': GO[0],
 'IV': IV[0],
 'TD': TD[0],
 'XZ': XZ[25],
 'ED': ED[23],
 'VF': VF[9],
 'DS': DS[0],
 'LU': LU[0],
 'TR': TR[0],
 'WO': WO[0],
 'KR': KR[8],
 'VL': VL[0],
 'CI': CI[0],
 'JJ': JJ[0],
 'TQ': TQ[0],
 'HN': HN[0],
 'YG': YG[0],
 'AA': AA[0],
 'MC': MC[24],
 'MI': MI[0],
 'EE': EE[0],
 'TH': TH[0],
 'XM': XM[0],
 'AF': AF[7],
 'JL': JL[0],
 'XE': XE[0],
 'XP': XP[0],
 'ZF': ZF[0],
 'EM': EM[4],
 'EI': EI[0],
 'AW': AW[0],
 'BJ': BJ[0],
 'NL': NL[0],
 'KF': KF[0],
 'NE': NE[0],
 'TY': TY[0],
 'KB': KB[0],
 'WP': WP[0],
 'XL': XL[21],
 'PK': PK[11],
 'JF': JF[0],
 'EX': EX[22],
 'ZZ': ZZ[0],
 'SL': SL[0],
 'JS': JS[3],
 'HP': HP[0],
 'DI': DI[12],
 'WK': WK[0]}

In [97]:
for node in valves.values():
    for neigh in node.neighbors:
        G.add_edge(node,neigh)

paths = nx.shortest_path(G)
# [x for x in paths[valves['AA']][valves['DD']]]

In [98]:
# #just to test

# time = 30
# total_pressure = 0
# opened_valves_names = []

# current_valve_name = 'AA'
# current_valve = valves[current_valve_name]
# pressure_gains = {}
# for k,v in valves.items():
#     #find best next step
#     if (k != current_valve) and (k not in opened_valves_names):
#         node_distance = (len(paths[current_valve][v])-1)
#         node_gain = (time-node_distance)*v.rate
#         if node_gain <= 0:
#             continue
#         if node_gain not in pressure_gains.items():
#             pressure_gains[node_gain] = [(node_distance+1, v)]
#         else:
#             pressure_gains[node_gain].append((node_distance+1, v))

# pressure_gains

In [101]:
def find_solution(paths,time,total_pressure,opened_valves_names):
    current_valve_name = opened_valves_names[-1]
    # print(f'visited: {opened_valves_names}; {total_pressure=}; {time=}')
    if time == 0:
        return total_pressure
    elif time < 0:
        return -1
    
    current_valve = valves[current_valve_name]
    pressure_gains = {}
    for k,v in valves.items():
        #find the next step
        if (k != current_valve) and (k not in opened_valves_names):
            node_distance = (len(paths[current_valve][v])-1)
            node_gain = (time-node_distance-1)*v.rate
            if node_gain <= 0:
                continue
            if node_gain not in pressure_gains.items():
                pressure_gains[node_gain] = [(node_distance+1, v)]
            else:
                pressure_gains[node_gain].append((node_distance+1, v))
    
    if len(pressure_gains):
        #search recursively
        best_pressure = -1
        for pressure,nodes in pressure_gains.items():
            for used_time, node in nodes:
                tmp_pressure_gain = find_solution(paths,time-used_time,total_pressure+pressure,opened_valves_names+[node.name])
                if best_pressure < tmp_pressure_gain:
                    best_pressure = tmp_pressure_gain
        return best_pressure
    else:
        return total_pressure

time = 30
total_pressure = 0
opened_valves_names = []
find_solution(paths,time,total_pressure,['AA'])

# find_solution(paths,time,total_pressure,['AA', 'DD', 'BB', 'JJ', 'HH'])

1991

# Second part

In [105]:
global ans_2
ans_2 = 0

def find_solution(paths,time_1,time_2,total_pressure,opened_valves_names,current_valve_name_1,current_valve_name_2):
    global ans_2

    if (time_1 == 0 and time_2 == 0) or len(opened_valves_names) == len(valves):
        if ans_2 < total_pressure:
            print(f'new best solution: {total_pressure}')
            ans_2 = total_pressure
        return total_pressure
    elif time_1 < 0 or time_2 < 0:
        print(f'new best solution: {total_pressure}, {time_1=}, {time_2=}')
        return -1
    
    current_valve_1 = valves[current_valve_name_1]
    pressure_gains_1 = []

    for k,v in valves.items():
        #find the next step
        if k not in opened_valves_names:
            node_distance = (len(paths[current_valve_1][v])-1)
            node_gain = (time_1-node_distance-1)*v.rate
            if node_gain <= 0:
                continue
            pressure_gains_1.append((node_gain,node_distance+1, v))

    current_valve_2 = valves[current_valve_name_2]
    pressure_gains_2 = []

    for k,v in valves.items():
        #find the next step
        if k not in opened_valves_names:
            node_distance = (len(paths[current_valve_2][v])-1)
            node_gain = (time_2-node_distance-1)*v.rate
            if node_gain <= 0:
                continue
            pressure_gains_2.append((node_gain,node_distance+1, v))

    pressure_gains_1.sort(key=lambda x:x[0], reverse=True)
    pressure_gains_2.sort(key=lambda x:x[0], reverse=True)
    
    if len(pressure_gains_1) == 1 and len(pressure_gains_2) == 1 and pressure_gains_1 == pressure_gains_2:
        # print('Take care of this case!')
        # print(f'{pressure_gains_1=}, {pressure_gains_2=}')
        pressure_1,used_time_1, node_1 = pressure_gains_1[0]
        pressure_2,used_time_2, node_2 = pressure_gains_2[0]
        
        tmp_pressure_gain_1 = find_solution(paths,time_1-used_time_1,time_2,total_pressure+pressure_1,opened_valves_names+[node_1.name],node_1.name,current_valve_name_2)
        tmp_pressure_gain_2 = find_solution(paths,time_1,time_2-used_time_2,total_pressure+pressure_2,opened_valves_names+[node_2.name],current_valve_name_1,node_2.name)
        if tmp_pressure_gain_1 > tmp_pressure_gain_2:
            return tmp_pressure_gain_1
        else:
            return tmp_pressure_gain_2

    elif len(pressure_gains_1):
        #search recursively
        best_pressure = -1
        for pressure_1,used_time_1, node_1 in pressure_gains_1:
            for pressure_2,used_time_2, node_2 in pressure_gains_2:
                if node_1 == node_2:
                    continue
                if len(opened_valves_names) == 1:
                    print(f'testing: 1st going to {node_1.name}, 2nd going to {node_2.name}')
                tmp_pressure_gain = find_solution(paths,time_1-used_time_1,time_2-used_time_2,total_pressure+pressure_1+pressure_2,opened_valves_names+[node_1.name,node_2.name],node_1.name,node_2.name)
                if best_pressure < tmp_pressure_gain:
                    best_pressure = tmp_pressure_gain
        if ans_2 < total_pressure:
            print(f'{datetime.datetime.now()} new best solution: {total_pressure}')
            ans_2 = total_pressure
        # if best_pressure == -1:
        #     print('here')
        return best_pressure
    elif len(pressure_gains_2):
        #search recursively
        best_pressure = -1
        pressure_1,used_time_1, node_1 = 0, 0, valves[current_valve_name_1]
        for pressure_2,used_time_2, node_2 in pressure_gains_2:
            if node_1 == node_2:
                continue
            if len(opened_valves_names) == 1:
                print(f'testing: 1st going to {node_1.name}, 2nd going to {node_2.name}')
            tmp_pressure_gain = find_solution(paths,time_1-used_time_1,time_2-used_time_2,total_pressure+pressure_1+pressure_2,opened_valves_names+[node_1.name,node_2.name],node_1.name,node_2.name)
            if best_pressure < tmp_pressure_gain:
                best_pressure = tmp_pressure_gain
        if ans_2 < total_pressure:
            print(f'{datetime.datetime.now()} new best solution: {total_pressure}')
            ans_2 = total_pressure
        # if best_pressure == -1:
        #     print('here')
        return best_pressure
    else:
        if ans_2 < total_pressure:
            print(f'{pressure_gains_1=}, {pressure_gains_2=}, ')
            print(f'{datetime.datetime.now()} new best solution: {total_pressure}')
            ans_2 = total_pressure
        return total_pressure

time = 26
total_pressure = 0
opened_valves_names = []

# find_solution(paths,time-2,time,480,['AA','DD'],'DD','AA')

find_solution(paths,time,time,0,['AA'],'AA','AA')


testing: 1st going to MC, 2nd going to EX
2022-12-16 03:03:19.970433 new best solution: 2265
pressure_gains_1=[], pressure_gains_2=[], 
2022-12-16 03:03:19.971119 new best solutionNNNN: 2277
2022-12-16 03:03:19.971770 new best solution: 2283
pressure_gains_1=[], pressure_gains_2=[], 
2022-12-16 03:03:19.974265 new best solutionNNNN: 2300
pressure_gains_1=[], pressure_gains_2=[], 
2022-12-16 03:03:20.020887 new best solutionNNNN: 2478
pressure_gains_1=[], pressure_gains_2=[], 
2022-12-16 03:03:20.021079 new best solutionNNNN: 2517
pressure_gains_1=[], pressure_gains_2=[], 
2022-12-16 03:03:20.250154 new best solutionNNNN: 2653
pressure_gains_1=[], pressure_gains_2=[], 
2022-12-16 03:03:23.822443 new best solutionNNNN: 2705
testing: 1st going to MC, 2nd going to ED
testing: 1st going to MC, 2nd going to XZ
testing: 1st going to MC, 2nd going to XL
testing: 1st going to MC, 2nd going to FX
testing: 1st going to MC, 2nd going to RW
testing: 1st going to MC, 2nd going to PK
testing: 1st goi

KeyboardInterrupt: 